In [3]:
from __future__ import annotations
from dataclasses import dataclass, field
from typing import Dict, List
from uuid import uuid4

class InventoryError(Exception):
    pass

@dataclass
class Product:
    name: str
    price: float
    stock: int = 0
    _pid: str = field(default_factory=lambda: str(uuid4()), init=False, repr=False, compare=False)

    def update_stock(self, quantity: int) -> None:
        new_stock = self.stock + quantity
        if new_stock < 0:
            raise InventoryError(
                f"Недостаточно товара '{self.name}': есть {self.stock}, нужно {abs(quantity)}."
            )
        self.stock = new_stock

    def __hash__(self) -> int:
        return hash(self._pid)

    def __eq__(self, other: object) -> bool:
        return isinstance(other, Product) and self._pid == other._pid

    def __repr__(self) -> str:
        return f"Product(name={self.name!r}, price={self.price:.2f}, stock={self.stock})"


@dataclass
class Order:
    products: Dict[Product, int] = field(default_factory=dict)

    def add_product(self, product: Product, quantity: int = 1) -> None:
        if quantity <= 0:
            raise ValueError("quantity должен быть > 0")
        product.update_stock(-quantity)
        self.products[product] = self.products.get(product, 0) + quantity

    def remove_product(self, product: Product, quantity: int = 1) -> None:
        if quantity <= 0:
            raise ValueError("quantity должен быть > 0")
        if product not in self.products:
            raise KeyError(f"Товара '{product.name}' нет в заказе")
        in_order = self.products[product]
        if quantity > in_order:
            raise ValueError(
                f"В заказе только {in_order} шт. '{product.name}', нельзя удалить {quantity}."
            )
        product.update_stock(+quantity)
        left = in_order - quantity
        if left == 0:
            del self.products[product]
        else:
            self.products[product] = left

    def return_product(self, product: Product, quantity: int = 1) -> None:
        self.remove_product(product, quantity)

    def calculate_total(self) -> float:
        return sum(p.price * qty for p, qty in self.products.items())

    def __repr__(self) -> str:
        lines = ["Order("]
        for p, q in self.products.items():
            lines.append(f"  {p.name} x{q} @ {p.price:.2f} = {p.price*q:.2f}")
        lines.append(f"  Итого: {self.calculate_total():.2f}")
        lines.append(")")
        return "\n".join(lines)


@dataclass
class Store:
    products: List[Product] = field(default_factory=list)

    def add_product(self, product: Product) -> None:
        self.products.append(product)

    def list_products(self) -> List[str]:
        return [f"{p.name}: {p.price:.2f} ₽ — {p.stock} шт." for p in self.products]

    def create_order(self) -> Order:
        return Order()


store = Store()
p1 = Product("Ноутбук", 120000.0, stock=10)
p2 = Product("Мышь", 1500.0, stock=50)
p3 = Product("Клавиатура", 4500.0, stock=20)
for p in (p1, p2, p3):
    store.add_product(p)

print("Товары в магазине:")
for row in store.list_products():
    print(" -", row)

order = store.create_order()
order.add_product(p1, 1)
order.add_product(p2, 2)

print("\nПосле добавления в заказ:")
print(order)

print("\nСклад после резервирования:")
for row in store.list_products():
    print(" -", row)

order.remove_product(p2, 1)

print("\nПосле частичного удаления:")
print(order)

print("\nСклад после удаления (возврат на полку):")
for row in store.list_products():
    print(" -", row)

try:
    order.return_product(p3, 1)
except Exception as e:
    print("\nОжидаемая ошибка при возврате отсутствующего товара:", e)

print("\nИтог к оплате:", order.calculate_total(), "₽")

Товары в магазине:
 - Ноутбук: 120000.00 ₽ — 10 шт.
 - Мышь: 1500.00 ₽ — 50 шт.
 - Клавиатура: 4500.00 ₽ — 20 шт.

После добавления в заказ:
Order(
  Ноутбук x1 @ 120000.00 = 120000.00
  Мышь x2 @ 1500.00 = 3000.00
  Итого: 123000.00
)

Склад после резервирования:
 - Ноутбук: 120000.00 ₽ — 9 шт.
 - Мышь: 1500.00 ₽ — 48 шт.
 - Клавиатура: 4500.00 ₽ — 20 шт.

После частичного удаления:
Order(
  Ноутбук x1 @ 120000.00 = 120000.00
  Мышь x1 @ 1500.00 = 1500.00
  Итого: 121500.00
)

Склад после удаления (возврат на полку):
 - Ноутбук: 120000.00 ₽ — 9 шт.
 - Мышь: 1500.00 ₽ — 49 шт.
 - Клавиатура: 4500.00 ₽ — 20 шт.

Ожидаемая ошибка при возврате отсутствующего товара: "Товара 'Клавиатура' нет в заказе"

Итог к оплате: 121500.0 ₽
